In [ ]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

unprocessed = pd.read_excel('../Datasets/Oneil_combination_response.xls')
unprocessed['ic50'] = unprocessed['X/X0'].apply(lambda x: 1 if x>=.45 and x<=0.55 else 0 )
unprocessed['new drugA Conc (µM)'] = np.log2(unprocessed['drugA Conc (µM)'])
unprocessed['new drugB Conc (µM)'] = np.log2(unprocessed['drugB Conc (µM)'])

/home/mohammad/Documents/ComplexMatrixComb/Codes/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Creating Drug-cell line matrix

In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def process_drug_combination_data(
    df,
    cell_line_col='cell_line',
    combo_name_col='combination_name',
    drugA_col='drugA Conc (µM)',
    drugB_col='drugB Conc (µM)',
    new_drugA_col='new drugA Conc (µM)',
    new_drugB_col='new drugB Conc (µM)',
    ic50_col='ic50',
    threshold=0.5,
    random_state=None
):
    """
    Process drug combination data to create complex concentration matrices.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe containing drug combination data
    cell_line_col : str
        Column name for cell line identifiers
    combo_name_col : str
        Column name for combination identifiers
    drugA_col, drugB_col : str
        Columns for original drug concentrations
    new_drugA_col, new_drugB_col : str
        Columns for transformed drug concentrations
    ic50_col : str
        Column indicating IC50 values (1 = active)
    threshold : float (0-1)
        Threshold for dropping columns with too many missing values
    random_state : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    tuple of (pd.DataFrame, pd.DataFrame)
        Returns two dataframes:
        1. Processed matrix with new concentrations
        2. Original concentration matrix
    """
    
    # Create mappings for drug concentrations
    active_drugs = df[df[ic50_col] == 1]
    l1 = active_drugs[drugA_col].value_counts().index.tolist()
    l2 = active_drugs[drugB_col].value_counts().index.tolist()
    
    l1_mapping = {value: idx for idx, value in enumerate(l1)}
    l2_mapping = {value: idx for idx, value in enumerate(l2)}
    
    # Create all possible cell line × combination pairs
    names = pd.merge(
        df[cell_line_col].drop_duplicates(),
        df[combo_name_col].drop_duplicates(),
        how='cross'
    )
    
    # Initialize result dictionaries
    new_conc_matrix = {}
    orig_conc_matrix = {}
    
    # Process each cell line and combination
    for _, row in tqdm(names.iterrows(), total=len(names)):
        cell_line = row[cell_line_col]
        combo = row[combo_name_col]
        
        if cell_line not in new_conc_matrix:
            new_conc_matrix[cell_line] = []
            orig_conc_matrix[cell_line] = []
        
        # Filter relevant data
        mask = (
            (df[cell_line_col] == cell_line) & 
            (df[combo_name_col] == combo) & 
            (df[ic50_col] == 1)
        )
        temp = df[mask]
        
        if temp.empty:
            new_conc_matrix[cell_line].append(np.nan)
            orig_conc_matrix[cell_line].append(np.nan)
        else:
            # Sort by drug concentrations
            temp['sorting_key'] = temp.apply(
                lambda x: (l1_mapping[x[drugA_col]], l2_mapping[x[drugB_col]]), 
                axis=1
            )
            temp = temp.sort_values('sorting_key').iloc[0]
            
            # Store as complex numbers
            new_conc_matrix[cell_line].append(
                complex(temp[new_drugA_col], temp[new_drugB_col])
            )
            orig_conc_matrix[cell_line].append(
                complex(temp[drugA_col], temp[drugB_col])
            )
    
    # Convert to dataframes
    cols = unprocessed['combination_name'].unique().tolist()
    
    new_df = pd.DataFrame.from_dict(new_conc_matrix, orient='index', columns=cols)
    orig_df = pd.DataFrame.from_dict(orig_conc_matrix, orient='index', columns=cols)
    
    # Apply threshold filtering
    threshold_count = int(len(new_df) * threshold)
    
    new_df = new_df.dropna(thresh=threshold_count, axis=1).fillna(0)
    orig_df = orig_df.dropna(thresh=threshold_count, axis=1).fillna(0)
    
    return new_df, orig_df


# Example usage:
# new_conc, orig_conc = process_drug_combination_data(unprocessed)
# new_conc.to_csv('drug_cell_log_matrix.csv')
# orig_conc.to_csv('drug_cell_matrix.csv')

100%|██████████| 583/583 [00:02<00:00, 247.99it/s]


In [20]:
a

,5-FU & ABT-888,5-FU & BEZ-235,5-FU & Bortezomib,5-FU & Dasatinib,5-FU & L778123,5-FU & geldanamycin,5-FU & Lapatinib,5-FU & PD325901,5-FU & AZD1775,5-FU & MK-2206,...,Zolinza & MK-8669,Zolinza & Oxaliplatin,Zolinza & Dinaciclib,Zolinza & MK-8776,Zolinza & SN-38,Zolinza & Sorafenib,Zolinza & Sunitinib,Zolinza & Erlotinib,Zolinza & Temozolomide,Zolinza & Topotecan
A2058,0.0,3.321928-5.643856j,0.0,3.321928+1.584963j,3.321928+2.321928j,3.321928-3.321928j,3.321928+2.321928j,3.321928-3.321928j,3.321928-2.321928j,3.321928+2.000000j,...,0.0,0.0,0.0,-1.621488+0.201634j,0.201634-9.449769j,0.0,0.0,0.0,-1.621488+5.781360j,0.0


In [ ]:
l1 = unprocessed[unprocessed['ic50']==1]['drugA Conc (µM)'].value_counts().index.tolist()  # l1 values
l2 = unprocessed[unprocessed['ic50']==1]['drugB Conc (µM)'].value_counts().index.tolist()  # l2 values
l1_mapping = {value: position for position, value in enumerate(l1)}
l2_mapping = {value: position for position, value in enumerate(l2)}


names = pd.merge(unprocessed['cell_line'].drop_duplicates(), unprocessed['combination_name'].drop_duplicates(), how='cross')
result = dict()
result2 = dict()

for _, i in tqdm(names.iterrows()):
    if not i['cell_line'] in result:
        result[i['cell_line']]=list()
        result2[i['cell_line']]=list()
    temp = unprocessed[(unprocessed['cell_line']==i['cell_line'])&(unprocessed['combination_name']==i['combination_name'])&(unprocessed['ic50']==1)]
    if temp.empty:
        result[i['cell_line']].append(np.nan)
        result2[i['cell_line']].append(np.nan)
    else:
        temp['sorting_key'] = temp.apply(lambda row: (l1_mapping[row['drugA Conc (µM)']], l2_mapping[row['drugB Conc (µM)']]), axis=1)
        temp = temp.sort_values(by='sorting_key')
        temp = temp.iloc[0]
        #temp = temp.sample(1, random_state=42)
        result[i['cell_line']].append(complex(temp['new drugA Conc (µM)'], temp['new drugB Conc (µM)']))
        result2[i['cell_line']].append(complex(temp['drugA Conc (µM)'], temp['drugB Conc (µM)']))
        

In [ ]:
cols = names[names['cell_line']=='A2058']['combination_name'].to_list()
result = pd.DataFrame.from_dict(result, orient='index', columns=cols)
result
result.isna()
result.isnull().sum(axis=1) / result.shape[1]
result
percent_missing = result.isnull().sum() * 100 / len(result)
missing_value_df = pd.DataFrame({'column_name': result.columns,
                                 'percent_missing': percent_missing})
missing_value_df[missing_value_df['percent_missing']<50].sort_values('percent_missing', ascending=False)

In [ ]:
#drop columns with too many nulls
limitPer = len(result) * .50
result = result.dropna(thresh=limitPer, axis=1)
result = result.fillna(0)

ratings = result.to_numpy()
# compute the non-zero elements in the rating matrix
matrix_size = np.prod(ratings.shape)
interaction = np.flatnonzero(ratings).shape[0]
sparsity = 100 * (interaction / matrix_size)

print('dimension: ', ratings.shape)
print('sparsity: {:.1f}%'.format(sparsity))
ratings

In [ ]:
cols = names[names['cell_line']=='A2058']['combination_name'].to_list()
result2 = pd.DataFrame.from_dict(result2, orient='index', columns=cols)
result2
result2.isna()
result2.isnull().sum(axis=1) / result2.shape[1]
result2
percent_missing = result2.isnull().sum() * 100 / len(result2)
missing_value_df = pd.DataFrame({'column_name': result2.columns,
                                 'percent_missing': percent_missing})
missing_value_df[missing_value_df['percent_missing']<50].sort_values('percent_missing', ascending=False)
limitPer = len(result2) * .50
result2 = result2.dropna(thresh=limitPer, axis=1)
result2 = result2.fillna(0)


In [ ]:
result.to_csv('drug_cell_log_matrix.csv')
result2.to_csv('drug_cell_matrix.csv')

### Matrix Factorization

In [ ]:
data = pd.read_csv('drug_cell_log_matrix_simple.csv', index_col='Unnamed: 0')
main_concs = pd.read_csv('drug_cell_matrix.csv', index_col='Unnamed: 0')

In [ ]:
## different mf method 

import numpy as np
import numpy.linalg as la
from tqdm import tqdm

def pmf_with_bias(X, test, k, learning_rate, num_iterations, lambda_reg, sigma_sq):
    """
    Probabilistic Matrix Factorization with user and item bias using stochastic gradient descent for complex numbers.
    
    Arguments:
    X -- Input matrix of shape (m, n) with complex numbers
    test -- Test matrix of the same shape as X with complex numbers
    k -- Number of latent features
    learning_rate -- Learning rate for gradient descent
    num_iterations -- Number of iterations for the optimization
    lambda_reg -- Regularization strength
    sigma_sq -- Variance of the complex Gaussian distribution
    
    Returns:
    U -- Matrix of shape (m, k) representing the user latent factors (complex)
    V -- Matrix of shape (n, k) representing the item latent factors (complex)
    b_u -- Vector of shape (m,) representing the user bias terms (real)
    b_v -- Vector of shape (n,) representing the item bias terms (real)
    """
    m, n = X.shape

    # Initialize U, V, b_u, and b_v with random values
    U = np.random.normal(scale=1.0 / k, size=(m, k)).astype(complex)
    V = np.random.normal(scale=1.0 / k, size=(n, k)).astype(complex)
    b_u = np.random.normal(scale=1.0 / k, size= m).astype(complex) 
    b_v = np.random.normal(scale=1.0 / k, size=n).astype(complex)
    
    for iteration in range(num_iterations):
        for i in range(m):
            for j in range(n):
                if X[i,j]!=0:
                    prediction = np.dot(U[i, :], V[j, :].conj()) + b_u[i] + b_v[j]
                    error = X[i, j] - prediction
                    grad_U = -error * V[j, :] + lambda_reg * U[i, :]
                    grad_V = -error * U[i, :] + lambda_reg * V[j, :]
                    grad_b_u = -error + lambda_reg * b_u[i]
                    grad_b_v = -error + lambda_reg * b_v[j]

                    U[i, :] -= learning_rate * grad_U
                    V[j, :] -= learning_rate * grad_V
                    b_u[i] -= learning_rate * grad_b_u
                    b_v[j] -= learning_rate * grad_b_v

                    prediction = np.clip(prediction, -14, 9)

        if iteration % 10 == 0:
            print('Iteration : ' + str(iteration))
            print('MAE train error')
            mask = np.nonzero(X)
            print(np.mean(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask])))
            mask = np.nonzero(test)
            print('MAE test error')
            print(np.mean(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask])))

            print('MSE train error')
            mask = np.nonzero(X)
            print(np.mean(pow(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2)))
            mask = np.nonzero(test)
            print('MSE test error')
            print(np.mean(pow(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2)))

        
    return U, V, b_u, b_v

ratings = data.to_numpy(dtype='complex')
train = ratings.copy()
test = ratings.copy()

#u, v, b_u, b_v = kernel_matrix_factorization(train, test, 200, .0001, 1000, 0.001)
U, V, b_u, b_v = pmf_with_bias(train, test, 700, 0.001, 300, 0.0001, 0.01)

In [ ]:
from tqdm import tqdm
for i in range(0, 5):
    f = open(f'/home/abdollahimohammad/Downloads/final uni codes/test_index_fold_{i}.txt')
    test_index = f.readlines()
    temp = main_concs.to_numpy(dtype='complex')
    indexes = np.where(temp!=complex(0,0))
    test_indexes = []

    for j in tqdm(test_index):
        j = int(j.replace('\n', ''))
        val = complex(main_concs.iloc[indexes[0][j], indexes[1][j]])
        #test_indexes.append(unprocessed[(unprocessed['combination_name']==main_concs.columns[indexes[1][j]]) & (unprocessed['cell_line']==main_concs.index[indexes[0][j]]) & (unprocessed['drugA Conc (µM)']==val.real) & (unprocessed['drugB Conc (µM)']==val.imag)].index.values[0])
        test_indexes.extend(unprocessed[(unprocessed['combination_name']==main_concs.columns[indexes[1][i]]) & (unprocessed['cell_line']==main_concs.index[indexes[0][i]])].index.values)
        #print(main_concs.index[indexes[0][i]], main_concs.columns[indexes[1][i]])
        #print(complex(main_concs.iloc[indexes[0][i], indexes[1][i]]).real)
        #break
    test_df = unprocessed[unprocessed.index.isin(test_indexes)]
    test_df.to_csv(f'test_fold_{i}.csv', index=False)
    test_df = unprocessed[~unprocessed.index.isin(test_indexes)]
    test_df.to_csv(f'train_fold_{i}.csv', index=False)

In [ ]:
## final results based on nested cross validation 

## different mf method 
import re
import numpy as np
import numpy.linalg as la
from tqdm import tqdm

def pmf_with_bias(X, test, k, learning_rate, num_iterations, lambda_reg, sigma_sq):
    """
    Probabilistic Matrix Factorization with user and item bias using stochastic gradient descent for complex numbers.
    
    Arguments:
    X -- Input matrix of shape (m, n) with complex numbers
    test -- Test matrix of the same shape as X with complex numbers
    k -- Number of latent features
    learning_rate -- Learning rate for gradient descent
    num_iterations -- Number of iterations for the optimization
    lambda_reg -- Regularization strength
    sigma_sq -- Variance of the complex Gaussian distribution
    
    Returns:
    U -- Matrix of shape (m, k) representing the user latent factors (complex)
    V -- Matrix of shape (n, k) representing the item latent factors (complex)
    b_u -- Vector of shape (m,) representing the user bias terms (real)
    b_v -- Vector of shape (n,) representing the item bias terms (real)
    """
    m, n = X.shape

    # Initialize U, V, b_u, and b_v with random values
    U = np.random.normal(scale=1.0 / k, size=(m, k)).astype(complex)
    V = np.random.normal(scale=1.0 / k, size=(n, k)).astype(complex)
    b_u = np.random.normal(scale=1.0 / k, size= m).astype(complex) 
    b_v = np.random.normal(scale=1.0 / k, size=n).astype(complex)
    
    for iteration in range(num_iterations):
        for i in range(m):
            for j in range(n):
                if X[i,j]!=0:
                    prediction = np.dot(U[i, :], V[j, :].conj()) + b_u[i] + b_v[j]
                    error = X[i, j] - prediction
                    grad_U = -error * V[j, :] + lambda_reg * U[i, :]
                    grad_V = -error * U[i, :] + lambda_reg * V[j, :]
                    grad_b_u = -error + lambda_reg * b_u[i]
                    grad_b_v = -error + lambda_reg * b_v[j]

                    U[i, :] -= learning_rate * grad_U
                    V[j, :] -= learning_rate * grad_V
                    b_u[i] -= learning_rate * grad_b_u
                    b_v[j] -= learning_rate * grad_b_v

                    prediction = np.clip(prediction, -14, 9)

        if iteration % 10 == 0:
            print('Iteration : ' + str(iteration))
            #print('MAE train error')
            mask = np.nonzero(X)
            #print(np.mean(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask])))
            mask = np.nonzero(test)
            #print('MAE test error')
            mae_loss = np.mean(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]))
            #print(mae_loss)

            #print('MSE train error')
            mask = np.nonzero(X)
            #print(np.mean(pow(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2)))
            mask = np.nonzero(test)
            #print('MSE test error')
            mse_loss = np.mean(pow(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2))
            #print(mse_loss)

            pred = np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :]
            mask_pred = pred[mask]
            mask_test = test[mask]
            mask_test_real = list()
            mask_pred_real = list()
            error=0
            c=0
            for i in range(len(mask_test)):
                a = complex(np.power(2, mask_test[i].real), np.power(2, mask_test[i].imag))
                b = complex(np.power(2, mask_pred[i].real), np.power(2, mask_pred[i].imag))
                mask_test_real.append(a)
                mask_pred_real.append(b)
            
            mae_loss_transformed = np.mean(np.abs(np.array(mask_test_real) - np.array(mask_pred_real)))
            #print(f'converted MAE transformed : {mae_loss_transformed}')
            mse_loss_transformed = np.sqrt(np.mean(np.power(np.abs(np.array(mask_test_real) - np.array(mask_pred_real)), 2)))
            #print(f'converted MSE transformed : {mse_loss_transformed}')
    return U, V, b_u, b_v, mae_loss, mse_loss, mae_loss_transformed, mse_loss_transformed

for fold in range(5):
    print(f'training on fold : {fold}')
    #f = open(f'/home/abdollahimohammad/Downloads/final uni codes/cross_validation_output_fold_{fold}.txt')
    #params = eval(re.search(r'\[.*?\]', f.readlines()[-1]).group(0))
    train_index = np.loadtxt(f"./O'neil files/train_index_fold_{fold}.txt", dtype=int)
    test_index = np.loadtxt(f"./O'neil files/test_index_fold_{fold}.txt", dtype=int)

    #train_index = np.loadtxt(f'/home/abdollahimohammad/Downloads/final uni codes/train_index_fold_{fold}.txt', dtype=int)
    #test_index = np.loadtxt(f'/home/abdollahimohammad/Downloads/final uni codes/test_index_fold_{fold}.txt', dtype=int)
    ratings = data.to_numpy(dtype='complex')

    result_np = ratings.copy()
    indexes = np.argwhere(result_np != complex(0,0))

    print("Train Index: ", len(train_index))
    print("Test Index: ", len(test_index))

    test = np.zeros(ratings.shape, dtype=complex)
    train = np.zeros(ratings.shape, dtype=complex)
    for temp in train_index:
        train[tuple(indexes[temp])] = result_np[tuple(indexes[temp])]
    for temp in test_index:
        test[tuple(indexes[temp])] = result_np[tuple(indexes[temp])]
    #u, v, b_u, b_v = kernel_matrix_factorization(train, test, 200, .0001, 1000, 0.001)
    U, V, b_u, b_v, mae_loss, mse_loss, mae_loss_transformed, mse_loss_transformed = pmf_with_bias(train, test, 700, 0.001, 300, 0.0001, 0.01)
    print(f'final MAE : {mae_loss}')
    print(f'final MSE : {mse_loss}')
    print(f'final MAE transformed : {mae_loss_transformed}')
    print(f'final MSE transformed : {mse_loss_transformed}')
    t = np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :]
    c = 0
    for i in test_index:
        i = indexes[i]
        combs = unprocessed[(unprocessed['cell_line']== data.index[i[0]]) & (unprocessed['combination_name']== data.columns[i[1]])][['ic50', 'new drugA Conc (µM)', 'new drugB Conc (µM)', 'X/X0']]
        #print(t[i[0], i[1]])
        pred = t[i[0], i[1]]
        #print(pred)

        a = min(combs['new drugA Conc (µM)'].unique().tolist(), key=lambda t: abs(pred.real-t))
        b = min(combs['new drugB Conc (µM)'].unique().tolist(), key=lambda t: abs(pred.imag-t))
        #print(combs['new drugB Conc (µM)'].unique().tolist())
        #print('Nearest : ' + str(a) + '\t' + str(b))
        #print(combs[combs['ic50']==1])

        pred_conc = combs[(combs['new drugA Conc (µM)']==a) & (combs['new drugB Conc (µM)']==b)]
        if pred_conc[(pred_conc['X/X0'] >= .44) & (pred_conc['X/X0'] <= .56)].shape[0] > 0:
            c += 1
            continue

    print(f'validation accuracy : {c/len(test_index)}')
    print('\n')

In [17]:
unprocessed[unprocessed['cell_line']=='A2058']

,BatchID,cell_line,drugA_name,drugA Conc (µM),drugB_name,drugB Conc (µM),combination_name,viability1,viability2,viability3,viability4,mu/muMax,X/X0,ic50,new drugA Conc (µM),new drugB Conc (µM)
0,1,A2058,5-FU,0.35,ABT-888,0.3500,5-FU & ABT-888,0.97082,1.09014,0.94901,0.99627,0.99158,0.98840,0,-1.514573,-1.514573
1,1,A2058,5-FU,0.35,ABT-888,1.0800,5-FU & ABT-888,1.12351,0.17989,0.91679,0.96175,0.96820,0.95687,0,-1.514573,0.111031
2,1,A2058,5-FU,0.35,ABT-888,3.2500,5-FU & ABT-888,1.04343,0.87839,0.92156,0.90967,0.95525,0.93985,0,-1.514573,1.700440
3,1,A2058,5-FU,0.35,ABT-888,10.0000,5-FU & ABT-888,0.91930,0.78351,0.79530,0.78845,0.88154,0.84855,0,-1.514573,3.321928
4,1,A2058,5-FU,1.08,ABT-888,0.3500,5-FU & ABT-888,1.02865,0.93881,0.87908,0.99230,0.98220,0.97563,0,0.111031,-1.514573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364123,3,A2058,Temozolomide,55.00,MK-8776,4.0000,Temozolomide & MK-8776,0.04691,0.05106,0.05461,NaN,-0.81531,0.08074,0,5.781360,2.000000
364124,3,A2058,Temozolomide,250.00,MK-8776,0.0925,Temozolomide & MK-8776,0.28131,0.23808,0.21594,NaN,0.12422,0.29698,0,7.965784,-3.434403
364125,3,A2058,Temozolomide,250.00,MK-8776,0.3250,Temozolomide & MK-8776,0.07592,0.12392,0.08747,NaN,-0.48682,0.12730,0,7.965784,-1.621488
364126,3,A2058,Temozolomide,250.00,MK-8776,1.1500,Temozolomide & MK-8776,0.03684,0.02096,0.02628,NaN,-1.22063,0.04603,0,7.965784,0.201634


In [13]:
len(names[names['cell_line']=='A2058']['combination_name'].to_list())

583

In [12]:
names = pd.merge(unprocessed['cell_line'].drop_duplicates(), unprocessed['combination_name'].drop_duplicates(), how='cross')

In [ ]:
import re
import numpy as np
import numpy.linalg as la
from tqdm import tqdm

def pmf_with_bias(X: np.array, test: np.array, k: int, learning_rate: float, num_iterations: int, lambda_reg: float):
    """
    Probabilistic Matrix Factorization with user and item bias using stochastic gradient descent for complex numbers.
    
    Arguments:
    X -- Input matrix of shape (m, n) with complex numbers
    test -- Test matrix of the same shape as X with complex numbers
    k -- Number of latent features
    learning_rate -- Learning rate for gradient descent
    num_iterations -- Number of iterations for the optimization
    lambda_reg -- Regularization strength
    sigma_sq -- Variance of the complex Gaussian distribution
    
    Returns:
    U -- Matrix of shape (m, k) representing the user latent factors (complex)
    V -- Matrix of shape (n, k) representing the item latent factors (complex)
    b_u -- Vector of shape (m,) representing the user bias terms (real)
    b_v -- Vector of shape (n,) representing the item bias terms (real)
    """
    m, n = X.shape

    # Initialize U, V, b_u, and b_v with random values
    U = np.random.normal(scale=1.0 / k, size=(m, k)).astype(complex)
    V = np.random.normal(scale=1.0 / k, size=(n, k)).astype(complex)
    b_u = np.random.normal(scale=1.0 / k, size= m).astype(complex) 
    b_v = np.random.normal(scale=1.0 / k, size=n).astype(complex)
    
    for iteration in range(num_iterations):
        for i in range(m):
            for j in range(n):
                if X[i,j]!=0:
                    prediction = np.dot(U[i, :], V[j, :].conj()) + b_u[i] + b_v[j]
                    error = X[i, j] - prediction
                    grad_U = -error * V[j, :] + lambda_reg * U[i, :]
                    grad_V = -error * U[i, :] + lambda_reg * V[j, :]
                    grad_b_u = -error + lambda_reg * b_u[i]
                    grad_b_v = -error + lambda_reg * b_v[j]

                    U[i, :] -= learning_rate * grad_U
                    V[j, :] -= learning_rate * grad_V
                    b_u[i] -= learning_rate * grad_b_u
                    b_v[j] -= learning_rate * grad_b_v

                    prediction = np.clip(prediction, -14, 9)

        if iteration % 10 == 0:
            print('Iteration : ' + str(iteration))
            #print('MAE train error')
            mask = np.nonzero(X)
            #print(np.mean(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask])))
            mask = np.nonzero(test)
            #print('MAE test error')
            mae_loss = np.mean(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]))
            #print(mae_loss)

            #print('MSE train error')
            mask = np.nonzero(X)
            #print(np.mean(pow(abs(X[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2)))
            mask = np.nonzero(test)
            #print('MSE test error')
            mse_loss = np.mean(pow(abs(test[mask]-(np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :])[mask]), 2))
            #print(mse_loss)

            pred = np.dot(U, V.T)+b_u[:, np.newaxis]+b_v[np.newaxis, :]
            mask_pred = pred[mask]
            mask_test = test[mask]
            mask_test_real = list()
            mask_pred_real = list()
            error=0
            c=0
            for i in range(len(mask_test)):
                a = complex(np.power(2, mask_test[i].real), np.power(2, mask_test[i].imag))
                b = complex(np.power(2, mask_pred[i].real), np.power(2, mask_pred[i].imag))
                mask_test_real.append(a)
                mask_pred_real.append(b)
            
            mae_loss_transformed = np.mean(np.abs(np.array(mask_test_real) - np.array(mask_pred_real)))
            #print(f'converted MAE transformed : {mae_loss_transformed}')
            mse_loss_transformed = np.sqrt(np.mean(np.power(np.abs(np.array(mask_test_real) - np.array(mask_pred_real)), 2)))
            #print(f'converted MSE transformed : {mse_loss_transformed}')
    return U, V, b_u, b_v, mae_loss, mse_loss, mae_loss_transformed, mse_loss_transformed


def load_fold_indices(fold, base_path="./O'neil files/"):
    """Load train and test indices for a given fold"""
    train_path = f"{base_path}train_index_fold_{fold}.txt"
    test_path = f"{base_path}test_index_fold_{fold}.txt"
    return (
        np.loadtxt(train_path, dtype=int),
        np.loadtxt(test_path, dtype=int)
    )

def create_train_test_matrices(ratings, indexes, train_idx, test_idx):
    """Create train and test matrices from indices"""
    train = np.zeros_like(ratings, dtype=complex)
    test = np.zeros_like(ratings, dtype=complex)
    
    for temp in train_idx:
        train[tuple(indexes[temp])] = ratings[tuple(indexes[temp])]
    for temp in test_idx:
        test[tuple(indexes[temp])] = ratings[tuple(indexes[temp])]
    
    return train, test

def calculate_validation_accuracy(t, test_idx, indexes, unprocessed, data):
    """Calculate validation accuracy by comparing predictions to ground truth"""
    c = 0
    for i in test_idx:
        row, col = indexes[i]
        cell_line = data.index[row]
        combo_name = data.columns[col]
        pred = t[row, col]
        
        combs = unprocessed[
            (unprocessed['cell_line'] == cell_line) & 
            (unprocessed['combination_name'] == combo_name)
        ][['ic50', 'new drugA Conc (µM)', 'new drugB Conc (µM)', 'X/X0']]
        
        # Find nearest concentrations
        a = min(combs['new drugA Conc (µM)'].unique(), 
                key=lambda t: abs(pred.real - t))
        b = min(combs['new drugB Conc (µM)'].unique(), 
                key=lambda t: abs(pred.imag - t))
        
        # Check if prediction falls in valid range
        pred_conc = combs[
            (combs['new drugA Conc (µM)'] == a) & 
            (combs['new drugB Conc (µM)'] == b)]
        if pred_conc[(pred_conc['X/X0'] >= 0.44) & (pred_conc['X/X0'] <= 0.56)].shape[0] > 0:
            c += 1
            
    return c / len(test_idx)

def run_cross_validation(data, unprocessed, n_folds=5, pmf_params=None):
    """Run full cross-validation pipeline"""
    if pmf_params is None:
        pmf_params = {
            'n_features': 700,
            'learning_rate': 0.001,
            'n_epochs': 300,
            'reg_param': 0.0001        
            }
    
    ratings = data.to_numpy(dtype='complex')
    result_np = ratings.copy()
    indexes = np.argwhere(result_np != complex(0, 0))
    
    for fold in range(n_folds):
        print(f'\nTraining on fold: {fold}')
        
        # Load fold data
        train_idx, test_idx = load_fold_indices(fold)
        print(f"Train samples: {len(train_idx)}, Test samples: {len(test_idx)}")
        
        # Create train/test matrices
        train, test = create_train_test_matrices(ratings, indexes, train_idx, test_idx)
        
        # Run PMF with bias
        U, V, b_u, b_v, mae, mse, mae_t, mse_t = pmf_with_bias(
            train, test, 
            pmf_params['n_features'], 
            pmf_params['learning_rate'], 
            pmf_params['n_epochs'], 
            pmf_params['reg_param']
        )
        
        # Calculate predictions
        t = np.dot(U, V.T) + b_u[:, np.newaxis] + b_v[np.newaxis, :]
        
        # Calculate validation accuracy
        val_acc = calculate_validation_accuracy(t, test_idx, indexes, unprocessed, data)
        
        # Print results
        print(f'Final MAE: {mae:.4f}')
        print(f'Final MSE: {mse:.4f}')
        print(f'Final MAE (transformed): {mae_t:.4f}')
        print(f'Final MSE (transformed): {mse_t:.4f}')
        print(f'Validation accuracy: {val_acc:.4f}')

# Example usage:
# run_cross_validation(data, unprocessed, n_folds=5)